# Current Testing Battery

In [1]:
import sys
import statistics
import numpy
from itertools import product
from timeit import default_timer as timer
from tqdm import tqdm
sys.path.append("..")
import gillespy2
from scipy import stats

Error:No module named 'pyximport'


In [2]:
from gillespy2.basic_ssa_solver import BasicSSASolver
from gillespy2.ssa_c_solver import SSACSolver
from gillespy2.basic_root_solver import BasicRootSolver
from gillespy2.optimized_ssa_solver import OptimizedSSASolver
from gillespy2.example_models import *

In [3]:
modelList = [Example(), Trichloroethylene(), MichaelisMenten(), Schlogl()]

solverList = [BasicSSASolver, SSACSolver, BasicRootSolver, OptimizedSSASolver, OptimizedSSASolver()]
timingList = []

precompile_solver = True


for model, solver in product(modelList, solverList):
    medianList = []
    exterior_stats = []
    if precompile_solver and (solver == SSACSolver or isinstance(solver, SSACSolver)): #precompile SSACSolver to save time in repated tests
        solver = SSACSolver(model)
    for i in tqdm(range(30), desc = f'Model: {model.name}, Solver: {solver.name}'):
        standard_results = model.run()
        start = timer()
        test_results = model.run(solver=solver)
        stop = timer()
        medianList.append(stop-start)
        interior_stats = []
        for species in standard_results[0].keys():
            if solver in [BasicSSASolver, BasicRootSolver]:
                interior_stats.append(stats.ks_2samp(standard_results[0][species], test_results[species])[0])
            else:
                interior_stats.append(stats.ks_2samp(standard_results[0][species], test_results[0][species])[0])
                
        exterior_stats.append(statistics.median(interior_stats))
    median = statistics.median(medianList)
    timingList.append([model, solver, median])

Model: Trichloroethylene, Solver: SSACSolver: 100%|██████████| 30/30 [00:01<00:00, 16.28it/s]
Model: Trichloroethylene, Solver: Basic Root Solver: 100%|██████████| 30/30 [00:38<00:00,  1.28s/it]
Model: Trichloroethylene, Solver: CythonSSASolver: 100%|██████████| 30/30 [00:02<00:00, 12.58it/s]
Model: Trichloroethylene, Solver: CythonSSASolver: 100%|██████████| 30/30 [00:02<00:00, 13.00it/s]
Model: Michaelis_Menten, Solver: SSACSolver: 100%|██████████| 30/30 [00:01<00:00, 16.31it/s]
Model: Michaelis_Menten, Solver: Basic Root Solver: 100%|██████████| 30/30 [01:06<00:00,  2.22s/it]
Model: Schlogl, Solver: CythonSSASolver: 100%|██████████| 30/30 [00:01<00:00, 16.99it/s]
